In [1]:
from rdflib import Graph, URIRef, RDF, Namespace, Literal
from rdflib.namespace import XSD
import pandas as pd
import os
import json

from collections import namedtuple

FMEAOntData = namedtuple("FMEAOntData", [
    "path","graph","ont_iri", "class_prefix", "op_prefix", "dp_prefix"
])


def load_FMEA_Ont():

    # Dateipfade zur Quell- und Zielontologie
    #source_ontology_path = r"C:\Users\Alexander Verkhov\OneDrive\Dokumente\SkillOA KB\KB1_v0.53_letzterStand.ttl"
    ontology_path = r"C:\Users\Alexander Verkhov\OneDrive\Dokumente\MPA\Implementierung_MPA\Test\src\FMEA_KG.ttl"

    # Überprüfen, ob die Dateien existieren
    if not os.path.exists(ontology_path):
        raise FileNotFoundError(f"Die Datei {ontology_path} wurde nicht gefunden.")

    # Namespaces der Ontologie
    ont_iri = "http://www.semanticweb.org/FMEA_VDA_AIAG_2021/"
    class_iri= ont_iri + "class_"
    op_iri = ont_iri + "op_"
    dp_iri = ont_iri + "dp_"

    # Lade die Quellontologie
    graph = Graph()
    graph.parse(ontology_path, format="turtle")

    return FMEAOntData(
        ontology_path, graph, ont_iri, class_iri, op_iri, dp_iri
    )
FMEA_Data = load_FMEA_Ont()
print(FMEA_Data.class_prefix)
print(FMEA_Data.dp_prefix)
print(FMEA_Data.op_prefix)

http://www.semanticweb.org/FMEA_VDA_AIAG_2021/class_
http://www.semanticweb.org/FMEA_VDA_AIAG_2021/dp_
http://www.semanticweb.org/FMEA_VDA_AIAG_2021/op_


In [38]:
def getFailureModeParameters(self, interruptedSkill: str) -> str:
        """Gibt rows-JSON zurück: [{"potFM","FMParam","t","v"} ...]"""
        base_sep = '' if self.ont_iri.endswith(('#','/')) else '#'
        searchSkillIri = self.ont_iri + base_sep + interruptedSkill

        query = f"""
            PREFIX cl: <{self.class_prefix}>
            PREFIX op: <{self.op_prefix}>
            PREFIX dp: <{self.dp_prefix}>
            SELECT DISTINCT ?potFM ?FMParam
            WHERE {{
                ?potFM a cl:FailureMode ;
                       op:preventsFunction <{searchSkillIri}> ;
                       dp:hasFailureModeParams ?FMParam .
                <{searchSkillIri}> a cl:Function .
            }}
        """
        res = self.graph.query(query)
        output_lines = []
        for row in res:  # row ist rdflib.query.ResultRow
            potFM = str(row["potFM"])    # URIRef → str
            fmparam = str(row["FMParam"])
            # Platzhalter für Typ/Wert (falls später im KG abgelegt):
            #print(f"potFM: {potFM}    FMParam: {fmparam}")
            output_lines.append(potFM)
            output_lines.append(fmparam)

        return "\n".join(output_lines)

result_json = getFailureModeParameters(FMEA_Data, "TestSkill1")
# Wenn du NACH dem Call zusätzlich tabellarisch ausgeben willst:
print("FMParams")
print(result_json)

def getSystemreactionForFailureMode(self, FMIri: str) -> str:
    query = f"""
        PREFIX cl: <{self.class_prefix}>
        PREFIX op: <{self.op_prefix}>
        PREFIX dp: <{self.dp_prefix}>
        SELECT DISTINCT ?sysReact ?SysReactParams
        WHERE {{
            <{FMIri}> a cl:FailureMode .
            ?sysReact a cl:SystemReaction ;
                      op:reactsOnFailureMode <{FMIri}> ;
                      dp:hasSysReactParams ?SysReactParams .
        }}
    """
    res = self.graph.query(query)

    output_lines = []
    for row in res:
        sysR   = str(row["sysReact"])
        params = str(row["SysReactParams"])
        output_lines.append(sysR)
        output_lines.append(params)

    return "\n".join(output_lines)

result_string = getSystemreactionForFailureMode(FMEA_Data, "http://www.semanticweb.org/FMEA_VDA_AIAG_2021/ExFailureMode1")
print("Result")
print(result_string)

FMParams
http://www.semanticweb.org/FMEA_VDA_AIAG_2021/ExFailureMode1
{
  "rows": [
    { "id": "ns=4;s=OPCUA.bool1",  "t": "bool", "v": true }
  ]
}
http://www.semanticweb.org/FMEA_VDA_AIAG_2021/ÜÜÜ
{
  "rows": [
    { "id": "ns=4;s=OPCUA.Z1",  "t": "Int16", "v": 27 }
  ]
}
Result
http://www.semanticweb.org/FMEA_VDA_AIAG_2021/Ex_Sysreact1
{
  "rows": [
    { "step": 0, "g": "meta", "k": "jobId", "t": "nodeId", "v": "MAIN.fbJob" },
    { "step": 0, "g": "method", "k": "methodId", "t": "nodeId", "v": "MAIN.fbJob.M_Methode1" },
    { "step": 0, "g": "input",  "k": "x", "i": 0, "t": "int32", "v": 0},
    { "step": 0, "g": "output",  "k": "yOut", "i": 0, "t": "int32", "v": "119"}
  ]
}


In [2]:
def getMonitoringActionForFailureMode(self, FMIri: str) -> str:
        base_sep = '' if self.ont_iri.endswith(('#','/')) else '#'
        defaultIri = self.ont_iri + base_sep + "checkParameters"
        query = f"""
            PREFIX cl: <{self.class_prefix}>
            PREFIX op: <{self.op_prefix}>
            PREFIX dp: <{self.dp_prefix}>
            SELECT DISTINCT ?monAct ?monActParams
            WHERE {{
                <{FMIri}> a cl:FailureMode .
                ?monAct a cl:MonitoringAction ;
                        op:monitorsFailureMode <{FMIri}> ;
                        dp:hasMonActParams ?monActParams .
                FILTER( ?monAct != IRI("{defaultIri}") )
            }}
        """
        print(query)
        res = self.graph.query(query)

        output_lines = []
        for row in res:
            sysR   = str(row["monAct"])
            params = str(row["monActParams"])
            output_lines.append(sysR)
            output_lines.append(params)

        return "\n".join(output_lines)

result_string = getMonitoringActionForFailureMode(FMEA_Data, "http://www.semanticweb.org/FMEA_VDA_AIAG_2021/ExFailureMode2")
print("Result")
print(result_string)

NameError: name 'FMEA_Data' is not defined

In [ ]:
from typing import Sequence
from rdflib import Graph, URIRef, Namespace, Literal
from rdflib.namespace import RDF, XSD

FMEA_Data = load_FMEA_Ont()
print(FMEA_Data.class_prefix)

def ingestOccuredFailure(self,id: str,failureModeIRI: str |None,monActIRI: Sequence[str]|None,srIRI: str|None,   # akzeptiert tuple oder list
        lastSkillName: str,lastProcessName: str,summary: str,plcSnapshot: str) -> bool:
        CL = Namespace(self.class_prefix)
        OP = Namespace(self.op_prefix)
        DP = Namespace(self.dp_prefix)
        def _to_list(x):
            if x is None:
                return []
            if isinstance(x, str):
                return [x]
            try:
                return list(x)
            except TypeError:
                return [x]

        def _print_param(name, value):
            if isinstance(value, str) or value is None:
                print(f"{name}: {value}")
            else:
                items = _to_list(value)
                print(f"{name} (len={len(items)}):")
                for i, v in enumerate(items, 1):
                    print(f"  {i}: {v}")

        def _print_list(name, items):
            print(f"{name} (len={len(items)}):")
            for i, v in enumerate(items, 1):
                print(f"  {i}: {v}")

        # Debug: Eingaben ausgeben
        print("---PYTHON----")
        _print_param("id", id)
        _print_param("failureModeIRI", failureModeIRI)
        _print_param("monActIRI", monActIRI)
        _print_param("srIRI", srIRI)
        _print_param("lastSkillName", lastSkillName)
        _print_param("lastProcessName", lastProcessName)
        _print_param("summary", summary)
        _print_param("plcSnapshot", plcSnapshot)

        # Separator abhängig von self.ont_iri
        base_sep = '' if self.ont_iri.endswith(('#','/')) else '#'

        # Hilfsfunktion zum Erzeugen der neuen IDs
        def _make_ids(marker: str, n: int) -> list[str]:
            # i startet bei 1
            return [f"{self.ont_iri}{base_sep}{marker}{id}_{i}" for i in range(1, n + 1)]
        def _make_id_str(marker:str) ->str:
            return f"{self.ont_iri}{base_sep}{marker}{id}"
        
        CL = Namespace(self.class_prefix)
        OP = Namespace(self.op_prefix)
        DP = Namespace(self.dp_prefix)
        
        def insert_sr_and_fm(Occfm_id: str,*,lastSkill: str,snapShot: str,fm: str | None = None,summary_text: str | None = None,
                        Occsr_id: str | None = None,srIRI: str | None = None,
                        m_ids: list[str] | None = None,mon_list: list[str] | None = None,) -> None:
            g = self.graph
            ofm = URIRef(Occfm_id)
            g.add((ofm, RDF.type, CL.OccuredFailure))
            if (Occfm_id.startswith("UFM_") or not fm):
                g.add((ofm, DP.isUnknownFailure, Literal(True)))
            else:
                g.add((ofm, OP.occuredWithStamp, URIRef(fm)))
            g.add((ofm, DP.hasOccuredFailureParams, Literal(snapShot)))
            g.add((ofm, OP.preventedFunction, URIRef(str(self.ont_iri + base_sep + lastSkill))))
            if summary_text:
                if summary_text != "":
                    g.add((ofm, DP.hasOccuredFailureSummary, Literal(summary_text)))
            if Occsr_id:
                if Occsr_id != "":
                    esr = URIRef(Occsr_id)
                    g.add((esr, RDF.type, CL.ExecutedSR))
                    g.add((esr, OP.executedBecauseOfFM, ofm))
                    g.add((URIRef(srIRI), OP.hasSRExecutionStamp, esr))

            if m_ids and mon_list:
                for i, m_id_str in enumerate(m_ids):
                    ema = URIRef(m_id_str)                  # ExecutedMonAct-Individual
                    g.add((ema, RDF.type, CL.ExecutedMonAct))
                    g.add((ema, OP.executedBecauseOfFM, ofm))
                    if i < len(mon_list) and mon_list[i]:
                        g.add((URIRef(mon_list[i]), OP.hasMExecutionStamp, ema))

            g.serialize(destination=self.path, format="turtle")

        # Längen bestimmen (robust gegen str vs. list/tuple)
        mon_list = _to_list(monActIRI)
        # Neue IRI-Listen erzeugen
        fm_id = _make_id_str("FM_") if (failureModeIRI is not None and failureModeIRI != "") else _make_id_str("UFM_")
        m_ids = _make_ids("M_", len(mon_list)) if mon_list else None
        sr_id = _make_id_str("SR_") if (srIRI is not None and srIRI != "") else None

        print("fm_id", fm_id)
        if m_ids: _print_list("m_ids", m_ids)
        if sr_id: print("sr_id", sr_id)
        print("---PYTHON----")
        kwargs = dict(
            lastSkill=lastSkillName,
            snapShot=plcSnapshot,
            fm=failureModeIRI if failureModeIRI else None,
            summary_text=summary if summary else None,
        )

        if sr_id:
            kwargs["Occsr_id"] = sr_id
        if srIRI:
            kwargs["srIRI"] = srIRI
            print ("SRIRI: ", srIRI)
        if m_ids and mon_list:
            kwargs["m_ids"] = m_ids
            kwargs["mon_list"] = mon_list

        # Aufruf – nur das, was es gibt, wird übergeben
        insert_sr_and_fm(fm_id, **kwargs)
        return True

# Test-Inputs aus deinem Debug-Output
id_ = "evD2-879848469290100_2025-09-15_18-55-19"
failureModeIRI = "http://www.semanticweb.org/FMEA_VDA_AIAG_2021/ExFailureMode1"
monActIRI = []  # (len=0)
srIRI = "http://www.semanticweb.org/FMEA_VDA_AIAG_2021/Ex_Sysreact1"
lastSkillName = "TestSkill1"
lastProcessName = "ExTechnProcess1"
summary = "SRDone: OK"
plcSnapshot = '''==InventorySnapshot=={"rows":[{"id":"ns=4;s=OPCUA.bool1","nodeClass":"Variable","t":"Boolean"},{"id":"ns=4;s=OPCUA.bool2","nodeClass":"Variable","t":"Boolean"},{"id":"ns=4;s=OPCUA.bool3","nodeClass":"Variable","t":"Boolean"},{"id":"ns=4;s=OPCUA.Z1","nodeClass":"Variable","t":"Int16"},{"id":"ns=4;s=OPCUA.Z2","nodeClass":"Variable","t":"Int16"},{"id":"ns=4;s=OPCUA.Z3","nodeClass":"Variable","t":"Int16"},{"id":"ns=4;s=OPCUA.TriggerD2","nodeClass":"Variable","t":"Boolean"},{"id":"ns=4;s=OPCUA.M1","nodeClass":"Variable","t":"Int16"},{"id":"ns=4;s=OPCUA.M2","nodeClass":"Variable","t":"Int16"},{"id":"ns=4;s=OPCUA.M3","nodeClass":"Variable","t":"Int16"},{"id":"ns=4;s=OPCUA.DiagnoseFinished","nodeClass":"Variable","t":"Boolean"},{"id":"ns=4;s=OPCUA.lastExecutedSkill","nodeClass":"Variable","t":"STRING (-> String)"},{"id":"ns=4;s=OPCUA.lastExecutedProcess","nodeClass":"Variable","t":"STRING (-> String)"},{"id":"ns=4;s=MAIN.fbJob","nodeClass":"Object","t":"-"},{"id":"ns=4;s=MAIN.fbJob.M_Methode1","nodeClass":"Method","t":"in: [Int32], out: [Int32]"}],"vars":[{"id":"OPCUA.bool3","t":"bool","v":false},{"id":"OPCUA.bool1","t":"bool","v":true},{"id":"OPCUA.DiagnoseFinished","t":"bool","v":false},{"id":"OPCUA.TriggerD2","t":"bool","v":true},{"id":"OPCUA.bool2","t":"bool","v":false},{"id":"OPCUA.lastExecutedProcess","t":"string","v":"ExTechnProcess1"},{"id":"OPCUA.lastExecutedSkill","t":"string","v":"TestSkill1"},{"id":"OPCUA.M2","t":"int16","v":0},{"id":"OPCUA.Z3","t":"int16","v":0},{"id":"OPCUA.Z1","t":"int16","v":0},{"id":"OPCUA.M3","t":"int16","v":0},{"id":"OPCUA.M1","t":"int16","v":0},{"id":"OPCUA.Z2","t":"int16","v":0}]}==InventorySnapshot=='''

ingestOccuredFailure(FMEA_Data,
    id_,
    failureModeIRI,
    monActIRI,
    srIRI,
    lastSkillName,
    lastProcessName,
    summary,
    plcSnapshot
)



In [1]:
# requirements: pip install rdflib
from rdflib import Graph, URIRef, Literal
from rdflib.namespace import RDFS
from typing import Optional

def generate_failure_modes_from_template(
    input_ttl_path: str,
    output_ttl_path: Optional[str] = None,
    template_local_name: str = "ExFailureMode1",
    new_local_prefix: str = "generated_ExFailureMode",
    count: int = 500,
    update_rdfs_label: bool = True,
) -> str:
    """
    Erzeugt 'count' neue Individuals, die alle Tripel (Properties) des Template-Individuals
    'template_local_name' übernehmen. Neue Ressourcen heißen new_local_prefix + i.

    - input_ttl_path: Pfad zur bestehenden TTL-Datei (z. B. 'FMEA_KG.ttl')
    - output_ttl_path: Zielpfad; wenn None, wird neben der Eingabedatei *_augmented.ttl geschrieben
    - template_local_name: z. B. 'ExFailureMode1'
    - new_local_prefix: z. B. 'generated_ExFailureMode'
    - count: Anzahl der neuen Individuals
    - update_rdfs_label: falls True, wird ein vorhandenes rdfs:label auf den neuen Namen gesetzt

    Rückgabewert: Pfad der geschriebenen TTL-Datei
    """
    g = Graph()
    g.parse(input_ttl_path, format="turtle")

    # 1) Template-URI im Graphen finden (per Localname/Fragment)
    def _localname(u: URIRef) -> str:
        s = str(u)
        if "#" in s:
            return s.split("#", 1)[1]
        return s.rsplit("/", 1)[-1]

    template_uri = None
    for s in set(g.subjects()):
        if isinstance(s, URIRef) and _localname(s) == template_local_name:
            template_uri = s
            break

    if template_uri is None:
        raise ValueError(f"Template-Individual '{template_local_name}' wurde im Graphen nicht gefunden.")

    # 2) Alle (predicate, object)-Paare des Templates sammeln
    template_props = list(g.predicate_objects(subject=template_uri))

    # 3) Basisprefix der URI ableiten, um neue URIs konsistent zu bilden
    #    (ersetzt den Localname/Fragmentteil am Ende durch den neuen Namen)
    template_str = str(template_uri)
    if "#" in template_str:
        base_uri = template_str.rsplit("#", 1)[0] + "#"
    else:
        base_uri = template_str.rsplit("/", 1)[0] + "/"

    # 4) Erzeugen und hinzufügen
    for i in range(1, count + 1):
        new_name = f"{new_local_prefix}{i}"
        new_uri = URIRef(base_uri + new_name)

        # falls es das Individual schon gibt, überspringen wir die Duplikaterzeugung
        if (new_uri, None, None) in g:
            continue

        for p, o in template_props:
            # rdfs:label auf den neuen Namen setzen (optional)
            if update_rdfs_label and p == RDFS.label:
                g.add((new_uri, p, Literal(new_name)))
            else:
                g.add((new_uri, p, o))

    # 5) Speichern
    if output_ttl_path is None:
        if input_ttl_path.lower().endswith(".ttl"):
            output_ttl_path = input_ttl_path[:-4] + "_augmented.ttl"
        else:
            output_ttl_path = input_ttl_path + "_augmented.ttl"

    g.serialize(destination=output_ttl_path, format="turtle")
    return output_ttl_path


# --- Beispielaufruf ---
out_file = generate_failure_modes_from_template(
     input_ttl_path=r"C:\Users\Alexander Verkhov\OneDrive\Dokumente\MPA\Implementierung_MPA\Test\src\FMEA_KG.ttl",
     output_ttl_path=r"C:\Users\Alexander Verkhov\OneDrive\Dokumente\MPA\Implementierung_MPA\Test\src\FMEA_KG_augmented_500_trip.ttl",
     template_local_name="ÜÜÜ",
     new_local_prefix="generated_ExFailureMode",
     count=500,
     update_rdfs_label=True,
 )
print("Geschrieben nach:", out_file)


Geschrieben nach: C:\Users\Alexander Verkhov\OneDrive\Dokumente\MPA\Implementierung_MPA\Test\src\FMEA_KG_augmented_500_trip.ttl
